<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>

# Forecasting of deaths from Heart Failure on medical measurement

# Lab 4. Model Development


The purpose of this lab is to master classification patients for machine learning models.

After completing this lab you will be able to:

1. preprocess (normilize and transform categorical data) and create DataSet
2. features selection
3. make classification of clients
4. visualize decision tree of classification model  


## Outline


* Materials and Methods
* General Part
  * Import Libraries
  * Load the Dataset
  * Data preparation
      * Data transformation
      * Encoding and Normalization
  * Features selection
      * Chi-Squared Statistic
      * Mutual Information Statistic
      * Feature Importance
      * Correlation Matrix with Heatmap
  * Decision tree 
      * Build model
      * Visualization of decision tree
  * Classification models
      * Extra Trees Classifier
      * Logistic regression 
* Authors


----


## Materials and Methods


The data that we are going to use for this is a subset of an open source The Heart Failure Prediction Dataset. https://www.kaggle.com/datasets/asgharalikhan/mortality-rate-heart-patient-pakistan-hospital.

> This dataset is public available for research.
Please include this citation if you plan to use this database:
The data contains complete history of heart patients (so data scientists from different parts of the world can work with it). The dataset is collected from Pakistan, Faisalabad hospital named institute of cardiology

In this lesson, we will try to give answers to a set of questions that may be relevant when analyzing heart failure data:

1. What are the most useful Python libraries for classification analysis?
2. How to transform category data?
3. How to create DataSet?
4. How to do features selection?
5. How to make, fit and visualize classification model?

In addition, we will make the conclusions for the obtained results of our classification analysis to predict mortality rate of patients with heart disease.


[Scikit-learn](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01) (formerly scikits.learn and also known as sklearn) is a free software machine learning library for the Python programming language. It features various classification, regression and clustering algorithms including support vector machines, random forests, gradient boosting, k-means and DBSCAN, and is designed to interoperate with the Python numerical and scientific libraries NumPy and SciPy.


In [ ]:
!conda install --yes scikit-learn==0.24.2
!conda install --yes python-graphviz

## Import Libraries


Import the libraries necessary to use in this lab. We can add some aliases to make the libraries easier to use in our code and set a default figure size for further plots. Ignore the warnings.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
plt.rcParams["figure.figsize"] = (8, 6)
# Data transformation
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
# Features Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif
# Classificators
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
# warnings deactivate
import warnings
warnings.filterwarnings('ignore')
import graphviz

Further specify the value of the `precision` parameter equal to 2 to display two decimal signs (instead of 6 as default).


In [ ]:
pd.options.display.float_format = '{:.2f}'.format

## Load the Dataset


We will use the same DataSet that we have saved in previous labs.


In [ ]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0MI1EN/clean_df_new.csv')
df.head(5)

In [ ]:
df.shape

As you can see DataSet consist 47 columns. Target column is 'Mortality'. Also DataSet consist 368 rows. In previous labs we investigated these columns.


<details>
<summary><b>Click to see attribute information</b></summary>

Input features (column names):

1. `Age Group` - patient age divided by groups (categorical)
2. `Marital Status` - married or single (categorical)
3. `Lifestyle` - does the patient have healthy lifestyle (boolean)
4. `Sleep` - does the patient sleep enough?(boolean)
5. `Category` paid or free treatment (categorical)
6. `Depression` - does patient feel depressed? (boolean)
7. `Hyperlipidemia` - an excess of lipids or fats in your blood (boolean)
8. `Smoking` - does the patient smoke? (boolean)
9. `Diabetes` - does the patient have diabetes? (binary) 
10. `HTN` - hypertension, also known as high blood pressure (boolean) 
11. `Allergies` - does the patient have allergies? (boolean)
12. `BP` - blood pressure (float, normalized) 
13. `Thrombolysis` - uses medications or a minimally invasive procedure to break up blood clots and prevent new clots from forming (binary) 
14. `BGR` - blood glucose level (int) 
15. `CPK` - creatine phosphokinase level (int)
16. `ESR` - erythrocyte sedimentation rate (int) 
17. `WBC` - white blood cells, also known as leukocytes (int) 
18. `RBC` - red blood cells, also known as erythrocytes (float) 
19. `Hemoglobin` - hemoglobin level (float) 
20. `MCH` - mean corpuscular hemoglobin or the average amount in each of red blood cells of a hemoglobin (float)
21. `MCHC` - mean corpuscular hemoglobin concentration (float)
22. `PlateletCount` - count of platelets or thrombocytes (int)
23. `Lymphocyte` - share of lymphocytes in blood (float)
24. `Monocyte` -  share of monocytes in blood (float)
25. `Eosinophil` - count of eosinophils (int)
26. `Others` - other diseases, that weren't mentioned (categorical)
27. `Diagnosis` - what is the patient's diagnosis? (float)
28. `Hypersensitivity` - does the patient have hypersensitivity? (boolean)
29. `Chest pain type` - patient's chest pain stage (int)
30. `Resting BP` - resting blood pressure (float)
31. `Serum cholesterol` - amount of total cholesterol in their blood (float)
32. `FBS` - fasting blood sugar > 120 mg/dl (binary)
33. `Resting electrocardiographic` - resting electrocardiographic results (0 = normal; 1 = having ST-T; 2 = hypertrophy) (int)
34. `Max heart rate` - patient's maximum heart rate achieved (int)
35. `Angina` - does the patient have exercise induced angina (binary)
36. `ST depression` - ST depression induced by exercise relative to rest (float)
37. `Slope` - the slope of the peak exercise ST segment (1 = upsloping; 2 = flat; 3 = downsloping) (int)
38. `Vessels num` - number of major vessels (0-3) colored by flourosopy (int)
39. `Thal` - 3 = normal; 6 = fixed defect; 7 = reversable defect (int)
40. `Num` -  diagnosis of heart disease (angiographic disease status) (int)
41. `Streptokinase` - used to dissolve blood clots that have formed in the blood vessels. Does the patient take it? (binary)
42. `SK React` - what is the reaction from streptokinase (categorical)
43. `Follow up` - number of patient's visiting time (int)
44. `Max heart rate-binned` - patient's maximum heart rate achieved - binned (from Lab2) (categorical)
45. `Gender-male` - is the patient male (from Lab2)? (binary)
46. `Locality-urban` - is the patient's locality urban (from Lab2)? (binary)

Output feature (desired target):

47. `Mortality` - did the patient die of heart failure? (binary)
</details>


Our goal is to create the model for predicting mortality caused by Heart Failure. To do this we must analize and prepare data for such type of model.


## Data preparation


### Data transformation


First of all we should investigate how pandas recognized types of features


In [ ]:
df.info()

As you can see all categorical features was recogized like object. We must change thair type on "categorical". 


In [ ]:
col_cat = list(df.select_dtypes(include=['object']).columns)
col_cat

Let's look at the dataset size.


In [ ]:
df.loc[:, col_cat] = df[col_cat].astype('category')
df.info()

To see the unical values of exact feature (column) we can use:


In [ ]:
df['Age Group'].unique()

As was signed earlier the dataset contains 368 objects (rows), for each of which 47 features are set (columns), including 1 target feature (y). 7 features are categorical. These data type of values cannot use for classification. We must transform it to int or float.
To do this we can use **[LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01)** and **[OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01)**. These functions can encode categorical features as an integer array.

First of all we separate DataSet on input and output(target) DataSets


In [ ]:
X = df.drop(['Mortality'], axis=1)  #input columns
y = df['Mortality']   #target column

### Encoding and Normalization


Than create list of categorical fields and transform thair values to int arrays: (Replace ##YOUR CODE GOES HERE## with your Python code.)


In [ ]:
col_cat = list(X.select_dtypes(include=['category']).columns)
oe = OrdinalEncoder()
oe.fit(X[col_cat])
X_cat_enc = oe.transform(X[col_cat])

In [ ]:
X_cat_enc

Than we must transform arrays back into DataFrame:


In [ ]:
X_cat_enc = pd.DataFrame(X_cat_enc)
X_cat_enc.columns = col_cat

Numerical fields can have different scale and can consists negative values. These will lead to round mistakes and exeptions for some AI methods. To avoid it these features must be normalized.

Let's create list of numerical fields and normilize it using by **[MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01)**


In [ ]:
col_num = list(X.select_dtypes(include=['float', 'int', 'bool']).columns)
scaler = MinMaxScaler(feature_range=(0, 1))
X_num_enc = scaler.fit_transform(X[col_num])

In [ ]:
X_num_enc

Like in previous case transform back obtained arrays into DataFrame


In [ ]:
X_num_enc = pd.DataFrame(X_num_enc)
X_num_enc.columns = col_num
X_num_enc

Than we should concatenate these DataFrames in one input DataFrame


In [ ]:
x_enc = pd.concat([X_cat_enc, X_num_enc], axis=1)
x_enc

Our target column is already normalized, so we don't need to encode it.


## Features selection


As was signed before input fields consists 46 features. Of course some of them are more significant for classification.

There are two popular feature selection techniques that can be used for categorical input data and a categorical (class) target variable.

They are:

* Chi-Squared Statistic.
* Mutual Information Statistic.

Let’s take a closer look at each in turn.

To do this we can use **[SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01)**


### Chi-Squared Statistic


Pearson’s chi-squared statistical hypothesis test is an example of a test for independence between categorical variables.

You can learn more about this statistical test in the tutorial:

[A Gentle Introduction to the Chi-Squared Test for Machine Learning](https://machinelearningmastery.com/chi-squared-test-for-machine-learning/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01)
The results of this test can be used for feature selection, where those features that are independent of the target variable can be removed from the dataset.

The scikit-learn machine library provides an implementation of the chi-squared test in the **[chi2()](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01#sklearn.feature_selection.chi2)** function. This function can be used in a feature selection strategy, such as selecting the top k most relevant features (largest values) via the SelectKBest class.

For example, we can define the SelectKBest class to use the chi2() function and select all (or most significant) features.


Apply SelectKBest class to extract top 10 best features


In [ ]:
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(x_enc, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

concat two dataframes for better visualization 


In [ ]:
featureScores = pd.concat([dfcolumns, dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

### Mutual Information Statistic


Mutual information from the field of information theory is the application of information gain (typically used in the construction of decision trees) to feature selection.

Mutual information is calculated between two variables and measures the reduction in uncertainty for one variable given a known value of the other variable.

[You can learn more about mutual information in the following tutorial.](https://machinelearningmastery.com/information-gain-and-mutual-information?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01)

The scikit-learn machine learning library provides an implementation of mutual information for feature selection via the **[mutual_info_classif()](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01#sklearn.feature_selection.mutual_info_classif)** function.

Like chi2(), it can be used in the SelectKBest feature selection strategy (and other strategies).


In [ ]:
bestfeatures = SelectKBest(score_func=mutual_info_classif, k=10)
fit = bestfeatures.fit(x_enc, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns, dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

As you can see these 2 function select different significant features.


### Feature Importance


You can get the feature importance of each feature of your DataFrame by using the feature importance property of the exact classification model.
Feature importance gives you a score for each feature of your data, the higher the score more important or relevant is the feature towards your output variable.
For example:
Feature importance is an inbuilt class that comes with **[Tree Based Classifiers](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01)**, we will be using Extra Tree Classifier for extracting the top 10 features for the dataset.


Let's create and fit the model:


In [ ]:
model = ExtraTreesClassifier()
model.fit(x_enc, y)

use inbuilt class feature_importances of tree based classifiers


In [ ]:
print(model.feature_importances_)

Let's transform it into Series and plot graph of feature importances for better visualization


In [ ]:
feat_importances = pd.Series(model.feature_importances_, index=x_enc.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

You can see that for Extra Tree Classifier impotance of features are different than in previous cases. It means that there are not exact rules for features selection. And their impotance strictly depedence on model.


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <h1>Question 1:</h1>
    <p>Plot graph of 5 least important features</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute


<details><summary>Click here for the solution</summary>

```python
feat_importances.nsmallest(5).plot(kind='barh')
plt.show()

```

</details>


### Correlation Matrix with Heatmap


Correlation states how the features are related to each other.
Correlation can be positive (increase in one value of feature increases the value of the other variable) or negative (increase in one value of feature decreases the value of the other variable)
Heatmap makes it easy to identify which features are most related to the other variable, we will plot heatmap of correlated features using the seaborn library.


In [ ]:
corrmat = x_enc.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(35,35))
g=sns.heatmap(x_enc[top_corr_features].corr(),annot=True,cmap="RdYlGn")

We have already removed strictly correlated columns, thus we can use all these features.


## Classification models


## Decision tree


### Build model


As shown, the previous methods have high accuracy. However, the biggest drawback is the inability to visualize or justify the decision.


Decision trees are a popular supervised learning method for a variety of reasons. Benefits of decision trees include that they can be used for both regression and classification, they don’t require feature scaling, and they are relatively easy to interpret as you can visualize decision trees. This is not only a powerful way to understand your model, but also to communicate how your model works. Consequently, it would help to know how to make a visualization based on your model.


A **[Decision Tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01)** is a supervised algorithm used in machine learning. It is using a binary tree graph (each node has two children) to assign for each data sample a target value. The target values are presented in the tree leaves. To reach to the leaf, the sample is propagated through nodes, starting at the root node. In each node a decision is made, to which descendant node it should go. A decision is made based on the selected sample’s feature. Decision Tree learning is a process of finding the optimal rules in each internal tree node according to the selected metric.


This metod allows also to calculate features impotance.
Let's calculate them. Choice best 10 of them. Refit the model and visualize decision tree.


In [ ]:
model = DecisionTreeClassifier()
model.fit(x_enc, y)
yhat = model.predict(x_enc)
accuracy = accuracy_score(y, yhat)
print('Accuracy: %.2f' % (accuracy*100))

Create user function that will calculate accuracy of defined classificator model:


In [ ]:
def model_ac(x, y, clf):
    model.fit(x, y)
    yhat = model.predict(x)
    accuracy = accuracy_score(y, yhat)
    return accuracy

Now let's create user function that will calculate features importance of defined classificator model. And let's create the variable, that contains features sorted by importance in descending order.


In [ ]:
def model_imp(x, y, model):
    feat_importances = pd.Series(model.feature_importances_, index=x.columns)
    return feat_importances.sort_values(ascending=False)
imp = model_imp(x_enc, y, model)
print(imp)

Plot graph of feature importances for better visualization


In [ ]:
imp.nlargest(10).plot(kind='barh')
plt.show()

Build plot that show accuracy of defined model dependence on numbers of input features.


In [ ]:
col = []
ac = []
for c in imp.index:
    col.append(c)
    ac.append(model_ac(x_enc[col], y, model))
    print('Input fields: ', len(col), 'Accuracy: %.2f' % (ac[-1]*100))
ac = pd.Series(ac)
ac.plot()

We can see that 5 features is enough to make 100% accuracy. So let's create list of this 5 features in order to use them for our next classification models.


In [ ]:
col = imp.nlargest(5).index
col

Let's refit the model on most important features


In [ ]:
X_most_imp = x_enc[col]
model.fit(X_most_imp, y)
yhat = model.predict(X_most_imp)
accuracy = accuracy_score(y, yhat)
print('Accuracy: %.2f' % (accuracy*100))

### Visualization of decision tree


Let's visualize decision tree.
There are some ways to do it. 


### _Text visualization_


In [ ]:
text_representation = tree.export_text(model)
print(text_representation)

You can save it into file:


In [ ]:
with open("decistion_tree.log", "w") as fout:
    fout.write(text_representation)

### _Plot tree_


You can plot tree using by two different way:


**[plot_tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01)** (slow render - this can take some time): 


In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(model,
               feature_names = col, 
               filled = True)

In [ ]:
fig.savefig('decision_tree.png')

Or you can use **[python-graphviz](https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01)** library. This is more fast function


In [ ]:
# conda install python-graphviz

In [ ]:
dot_data = tree.export_graphviz(model,
               feature_names = col,
                                filled=True)

After creation you can draw graph


In [ ]:
graph = graphviz.Source(dot_data, format="png") 
graph

And render it into file:


In [ ]:
graph.render("decision_tree_graphivz")

### Extra Trees Classifier


Let's create and fit ExtraTreesClassifier on most important features and calculate accuracy of classification:


In [ ]:
model = ExtraTreesClassifier()
model.fit(X_most_imp, y)

Evaluate the model for obtain predictions


In [ ]:
yhat = model.predict(X_most_imp)
print(yhat)

Evaluate accuracy: 


In [ ]:
accuracy = accuracy_score(y, yhat)
print('Accuracy: %.2f' % (accuracy*100))

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <h1>Question 2:</h1>
    <p>Create the variable, that contains features sorted by importance in descending order for Extra Tree model (using 5 most important features)</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute


<details><summary>Click here for the solution</summary>

```python
imp = model_imp(X_most_imp, y, model)
print(imp)

```

</details>


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <h1>Question 3:</h1>
    <p>Build plot that show accuracy of Extra Tree model dependence on numbers of input features (using 5 most important features).</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute


<details><summary>Click here for the solution</summary>

```python
col = []
ac = []
for c in imp.index:
    col.append(c)
    ac.append(model_ac(X_most_imp[col], y, model))
    print('Input fields: ', len(col), 'Accuracy: %.2f' % (ac[-1]*100))
ac = pd.Series(ac)
ac.plot()

```

</details>


### Logistic regression 


There are many different techniques for scoring features and selecting features based on scores; how do you know which one to use?

A robust approach is to evaluate models using different feature selection methods (and numbers of features) and select the method that results in a model with the best performance.

**[Logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0MI1EN2415-2022-01-01)** is a good model for testing feature selection methods as it can perform better if irrelevant features are removed from the model. We will use this model in absolutelly similar way like previous one.


In [ ]:
model = LogisticRegression(solver='lbfgs')
model.fit(X_most_imp, y)
yhat = model.predict(X_most_imp)
accuracy = accuracy_score(y, yhat)
print('Accuracy: %.2f' % (accuracy*100))

As we can see, accuracy of Logistic Regression model is lower (about 80%).


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <h1>Question 4:</h1>
    <p>Calculate accuracy of Logistic Regression model using all features.</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute


<details><summary>Click here for the solution</summary>

```python
model = LogisticRegression(solver='lbfgs')
model.fit(x_enc, y)
yhat = model.predict(x_enc)
accuracy = accuracy_score(y, yhat)
print('Accuracy: %.2f' % (accuracy*100))

```

</details>


## Conclusions


In this lab we learned to do preliminary data processing. In particular, change data types, normalize and process categorical data. It was shown how to make feature selection by different methods. Shows how to work with different classifiers. It was also shown how to visualize a decision tree.
As a result of lab it was shown how on the basis of a statistical database predict if the patient will die of a heart failure.

The accuracy of Decision Tree and Extra Tree classifiers was 100%. And the accuracy of Logistic Regression about 80%. 


### Thank you for completing this lab!

## Author

<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">Joseph Santarcangelo</a>

### Other Contributors

<a href="https://www.linkedin.com/in/mahdi-noorian-58219234/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">Mahdi Noorian PhD</a>

Bahare Talayian

Eric Xiao

Steven Dong

Parizad

Hima Vasudevan

<a href="https://www.linkedin.com/in/fiorellawever/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">Fiorella Wenver</a>

<a href="https:// https://www.linkedin.com/in/yi-leng-yao-84451275/ " target="_blank" >Yi Yao</a>.

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |


<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
